In [22]:
import pickle
import networkx as nx
import numpy as np

In [17]:
path = '/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Axolotls/Stage44.Immature_CMPN_XUT1/KNN/5.pkl'
with open(path, 'rb') as handle:
    G = pickle.load(handle)

path2 = '/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Axolotls/Stage44.Immature_CMPN_XUT1/KNN/10.pkl'
with open(path2, 'rb') as handle:
    G2 = pickle.load(handle)

In [14]:
# G.number_of_nodes()

In [15]:
G.nodes

NodeView(('AMEX60DD038502', 'AMEX60DD047169', 'AMEX60DD016343', 'AMEX60DD052120', 'AMEX60DD020311', 'AMEX60DD029966', 'AMEX60DD029926', 'AMEX60DD001331', 'AMEX60DD043243', 'AMEX60DD015190', 'AMEX60DD032108', 'AMEX60DD042749', 'AMEX60DD015307', 'AMEX60DD039736', 'AMEX60DD010282', 'AMEX60DD029963', 'AMEX60DD024272', 'AMEX60DD053480', 'AMEX60DD003045', 'AMEX60DD006176', 'AMEX60DD046985', 'AMEX60DD047845', 'AMEX60DD042493', 'AMEX60DD010090', 'AMEX60DD011733', 'AMEX60DD026859', 'AMEX60DD040171', 'AMEX60DD043373', 'AMEX60DD010813', 'AMEX60DD029013', 'AMEX60DD039907', 'AMEX60DD044272', 'AMEX60DD028745', 'AMEX60DD000806', 'AMEX60DD013534', 'AMEX60DD029705', 'AMEX60DD008783', 'AMEX60DD023994', 'AMEX60DD032684', 'AMEX60DD003832', 'AMEX60DD015050', 'AMEX60DD025630', 'AMEX60DD006055', 'AMEX60DD022526', 'AMEX60DD017083', 'AMEX60DD010367', 'AMEX60DD007843', 'AMEX60DD016499', 'AMEX60DD011670', 'AMEX60DD035913', 'AMEX60DD015106', 'AMEX60DD045225', 'AMEX60DD010598', 'AMEX60DD008749', 'AMEX60DD027826', 

In [25]:
np.array(list(G.edges)).reshape(1, -1)[0] != np.array(list(G2.edges)).reshape(1, -1)[0]

/tmp/ipykernel_2897476/2814900781.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.array(list(G.edges)).reshape(1, -1)[0] != np.array(list(G2.edges)).reshape(1, -1)[0]


True

In [29]:
np.sum(np.array(list(G2.edges)).reshape(1, -1)[0] != np.array(list(G2.edges)).reshape(1, -1)[0])

0

In [8]:
import glob

for i in glob.glob('/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/*'):
    # print("mv "+i+"/corr_2D.csv "+i+"/MST/")
    if 'corr_2D.csv' in [i.split('/')[-1] for i in glob.glob(i+"/*")]:
        print(i)

In [10]:
len(glob.glob('/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/*'))

165